In [1]:
import pandas as pd
import json
import time

In [2]:
with open('./train.json') as data_file:
    data = json.load(data_file)
raw_train = pd.DataFrame(data)

with open('./test.json') as data_file:
    data = json.load(data_file)
raw_test = pd.DataFrame(data)

raw_test = pd.DataFrame(data)

for item in raw_train.columns:
    print(item)

bathrooms
bedrooms
building_id
created
description
display_address
features
interest_level
latitude
listing_id
longitude
manager_id
photos
price
street_address


The id is used for identifying the listing in the competition. For simplicity, we reindex the data by integers.

In [3]:
_id = raw_train.index
raw_train = raw_train.reset_index()

We are going to train a popular model xgboost. We will start with only the numerical features.

In [4]:
col = ['bathrooms', 'bedrooms', 'latitude', 'longitude', 'price']
X = raw_train[col]
y = raw_train['interest_level'].apply(lambda x: 0 if x=='low' else 1 if x=='medium' else 2)

# Folding

To be able to evaluate our model, we use the package model_selection to split the indices into:
    - Training data (used to train the models)
    - Validation data (used to evaluate the models. We avoid using the term 'test data' to differentiate this from 
    the test data from Kaggle)

For the later usage in stacking, we also split the training data in 2 folds.

In [5]:
import numpy as np
import sklearn
from sklearn import model_selection

skf = model_selection.StratifiedKFold(n_splits=3)
folds = skf.split(X, y)
_, fold1 = folds.next()
_, fold2 = folds.next()
_, validation_idx = folds.next()

train_idx = np.concatenate([fold1, fold2])

In [9]:
print fold1
print fold2
print validation_idx

[    0     1     2 ..., 16867 16904 16919]
[16331 16332 16333 ..., 33063 33066 33070]
[32851 32853 32854 ..., 49349 49350 49351]


Below we initialize the paramters for xgboost

In [10]:
import xgboost as xgb

param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.02
param['max_depth'] = 6
param['silent'] = 1
param['num_class'] = 3
param['eval_metric'] = 'mlogloss'
param['min_child_weight'] = 3
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['seed'] = 321
num_rounds = 1000

In [13]:
start = time.time()
X_train = np.array(X.iloc[train_idx])
y_train = np.array(y.iloc[train_idx])
xgtrain = xgb.DMatrix(X_train, label = y_train)

clf = xgb.train(param, xgtrain, num_rounds)

X_validation = np.array(X.iloc[validation_idx])
y_validation = np.array(y.iloc[validation_idx])
xgvalidation = xgb.DMatrix(X_validation)
y_prob = clf.predict(xgvalidation)

print 'The log loss is: %.3f ' % sklearn.metrics.log_loss(y_validation, y_prob)
print 'Time elapsed: %.2f seconds ' % (time.time() - start)

The log loss is: 0.656 
Time elapsed: 33.13 seconds 


In [24]:
print X_train.shape

(32902, 5)


# Adding Text Data

## Clean the text
The code below cleans the "description" column in the dataset.

In [11]:
import re
import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')

def clean_text(sentence):
    sentence = sentence.encode('ascii', errors='replace')
    sentence = sentence.lower()
    sentence = re.sub('[^\w\s]', ' ', sentence) # removes punctuations
    sentence = re.sub('_', ' ', sentence) # removes punctuations
    sentence = re.sub('\d+', ' ', sentence) # removes digits
    cleaned = ' '.join([w for w in sentence.split() if not w in stop]) # remove stopwords
    cleaned = ' '.join([w for w, pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos == 'JJ')])
    # selecting only nounds and adjectives
    
    cleaned = ' '.join([w for w in cleaned.split() if not len(w) <= 2])
    # remove single lettered words and digits
    
    cleaned = cleaned.strip()
    return cleaned

In [12]:
start = time.time()
raw_train['cleaned_text'] = raw_train['description'].apply(lambda x: clean_text(x))
print 'Time elapsed: %.2f ' % (time.time() - start)

Time elapsed: 293.17 


# TF-IDF

To take the text data into account, we need to somehow encoding the categorical data features into 
numerical features. Dummification (or one-hot encoding) is a popular way for general categorical features.
For text data, we would use a much more efficient method: tf-idf.

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

n_features = 500
n_topics = 10
n_top_words = 20

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
text_train = np.array(raw_train.loc[train_idx, 'cleaned_text'])
tf = tf_vectorizer.fit_transform(text_train)
TF = TfidfTransformer()
tf_idf_train = TF.fit_transform(tf)
tf_idf_train = tf_idf_train.toarray()
X_train_des = np.concatenate([X_train, tf_idf_train], axis=1)


text_validation = np.array(raw_train.loc[validation_idx, 'cleaned_text'].replace(np.nan, ''))
tf = tf_vectorizer.transform(text_validation)
tf_idf_validation = TF.transform(tf)
tf_idf_validation = tf_idf_validation.toarray()
X_validation_des = np.concatenate([X_validation, tf_idf_validation], axis=1)

text_fold1 = np.array(raw_train.loc[fold1, 'cleaned_text'].replace(np.nan, ''))
text_fold2 = np.array(raw_train.loc[fold2, 'cleaned_text'].replace(np.nan, ''))

In [17]:
print text_fold1.shape
print text_fold2.shape
print text_validation.shape

(16451,)
(16451,)
(16450,)


Below we train the model with both the numerical feature we used before, and the new text data.
We see that the performance is improved with no surprise -- we took more feature into account.
However, the improvement takes a lot of time.



In [ ]:
start_ = time.time()
xgtrain_des = xgb.DMatrix(X_train_des, label=y_train)
clf = xgb.train(param, xgtrain_des, num_rounds)
xgvalidation_des = xgb.DMatrix(X_validation_des)
y_prob_des = clf.predict(xgvalidation_des)
print 'Time elapsed: %.2f seconds' % (time.time() - start_)
print 'The log loss is: %.3f' % sklearn.metrics.log_loss(y_validation, y_prob_des)

# Stacking models with resampling

In the demo above we saw a drawback of the tree-based models: inefficiency when dealing with large cardinality of 
categorical feature.

One thing we can do is to train some simpler models first on the text data, and then stack back with the numerical predictors.

# Using sparse matrix.

The text data contains a lot of zeros, one simple way to gain efficiency is to use the sparse matrices.

In [18]:
tf = tf_vectorizer.transform(text_train) # fold1 & 2
tf_idf_train = TF.transform(tf)

tf = tf_vectorizer.transform(text_fold1)
tf_idf_fold1 = TF.transform(tf)

tf = tf_vectorizer.transform(text_fold2)
tf_idf_fold2 = TF.transform(tf)

tf = tf_vectorizer.transform(text_validation)
tf_idf_validation = TF.transform(tf)

# Creating the new feature

The function below create the new predictors with 2 folds we described in the slides.

In [19]:
y_fold1 = np.array(y.iloc[fold1])
y_fold2 = np.array(y.iloc[fold2])

def get_2fold_stack(model):
    model.fit(tf_idf_fold1, y_fold1)
    new_fold2 = model.predict_proba(tf_idf_fold2)[:, :2]
    v1 = model.predict_proba(tf_idf_validation)[:, :2]  ### There is model

    model.fit(tf_idf_fold2, y_fold2)
    new_fold1 = model.predict_proba(tf_idf_fold1)[:, :2]
    v2 = model.predict_proba(tf_idf_validation)[:, :2] ### The model here is different
    
    return np.concatenate([new_fold1, new_fold2], axis=0), (v1+v2)/2

Below we add the predictor created by logistic regression. The performance is actualy much better than 
training with only numerical features, even though doesn't require as much time.

In [22]:
X_validation.shape

(16450, 5)

In [25]:
start_ = time.time()
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression()

f_logit_train, f_logit_validation = get_2fold_stack(logit) 
# f_logit_train: prediction from fold1& 2 concat.
# f_logit_validation: prediction from validation

X_train_stack = np.concatenate([X_train, f_logit_train], axis=1) # X_train: numerical features from fold1 & 2
xgtrain_stack = xgb.DMatrix(X_train_stack, label=y_train)
X_validation_stack = np.concatenate([X_validation, f_logit_validation], axis=1)
xgvalidation_stack = xgb.DMatrix(X_validation_stack)

clf = xgb.train(param, xgtrain_stack, num_rounds)
y_prob_stack = clf.predict(xgvalidation_stack)
print 'The log loss is: %.3f' % sklearn.metrics.log_loss(y_validation, y_prob_stack)
print 'Time elapsed: %.2f' % (time.time() - start_)

The log loss is: 0.641
Time elapsed: 26.99


In [ ]:
start_ = time.time()
from sklearn.neighbors import KNeighborsClassifier


logit1000 = LogisticRegression(C=1000)
f_logit1000_train, f_logit1000_validation = get_2fold_stack(logit1000)

knn10 = KNeighborsClassifier(n_neighbors=10)
f_knn10_train, f_knn10_validation = get_2fold_stack(knn10)

knn100 = KNeighborsClassifier(n_neighbors=100)
f_knn100_train, f_knn100_validation = get_2fold_stack(knn100)

In [27]:
from sklearn.decomposition import LatentDirichletAllocation

topic10 = LatentDirichletAllocation(n_topics=10)
f_topic10_train = topic10.fit_transform(tf_idf_train)
f_topic10_validation = topic10.fit_transform(tf_idf_validation)

/Library/Python/2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/Library/Python/2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [34]:
tf_idf_train.toarray()

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.1370008],
       ..., 
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]])